File Configuration

In [ ]:
# 'Detection' or 'Classification'
prediction_strategy = 'Detection'

# 'Raw', 'Smoothed' or 'Smoothed and Alligned'
data_prep_stage = 'Smoothed'

# 'Enabled' or 'Disabled'
postmodeling_renders = 'Disabled'

Libraries used:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from plyfile import PlyData

from statistics import mean

from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, precision_score, recall_score, accuracy_score
from sklearn.preprocessing import StandardScaler

from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K


Limiting the GPU memory growth

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

Dictionaries

In [ ]:
# scans with annotations and, per preparation stage, filepaths
scans = {}

# render settings per data preparation stage 
stage_specific_settings = {
    'Raw': {
        'depth': (50, 400),
    },
    'Smoothed': {
        'depth': (-0.5, 0.5),
    },
    'Smoothed and Alligned': {
        'depth': (-0.5, 0.5),
    },
}

# set default settings based on the chosen data preparation stage  
default_settings = stage_specific_settings[data_prep_stage]

Files in use

In [ ]:
# 221111_144114__145900to148050 - First scan ever used
# flightpath: from class 3 to class 2

scans['221111_144114__145900to148050'] = {
        'Raw':                      'volume/scanData/221111_144114__binary_onlylines145900to148050.ply',
        'Smoothed':                 'volume/scanData/6_flight_vibr_orig_binary.ply',
        'Smoothed and Alligned':    'volume/scanData/6_flight_comp_vibr_binary.ply',
        'Annotations': {
            (280, 560): 3,
            (590, 640): 2,
            (660, 700): 2,
            (740, 810): 2,
            (960, 1010): 2,
            (1140, 1200): 2,
            (1230, 1280): 2,
            (1380, 1400): 2,
            (1450, 1470): 2,
        },
    }

In [ ]:
# 221111_144020__11170to15130 - Unused due to high distortion
# flightpath: from class 3 to class 2

# scans['221111_144020__11170to15130'] = {
#     'Raw':                      'volume/scanData/221111_144020__binary_onlylines11170to15130raw.ply',
#     'Smoothed':                 'volume/scanData/221111_144020__binary_onlylines11170to15130smoothedbySLM.ply',
#     'Smoothed and Alligned':    'volume/scanData/221111_144020__binary_onlylines11170to15130smoothedbySLM_AL.ply',
#     'Annotations': {
#         (980, 1020): 2,
#         (1040, 1080): 2,
#         (1180, 1220): 2,
#         (1460, 1500): 2,
#         (1520, 1580): 2,
#         (1620, 1860): 3,
#     },
# }

In [ ]:
# 221111_144114__40900to45309 - Unused due to high distortion
# flightpath: from class 3 to class 2

# scans['221111_144114__40900to45309'] = {
#     'Raw':                      'volume/scanData/221111_144020__binary_onlylines40900to45309raw.ply',
#     'Smoothed':                 'volume/scanData/221111_144020__binary_onlylines40900to45309smoothedbySLM.ply',
#     'Smoothed and Alligned':    'volume/scanData/221111_144020__binary_onlylines40900to45309smoothedbySLM_AL.ply',
#     'Annotations': {
#         (200, 480): 3,
#         (520, 640): 2,
#         (660, 720): 2,
#         (920, 960): 2,
#         (1000, 1020): 2,
#         (1100, 1140): 2,
#         (1180, 1240): 2,
#         (1500, 1560): 2,
#     },
# }

In [ ]:
# 221111_144114__25660to28100 - A 2nd iteration scan - recently added
# flightpath: from class 3 to class 2

scans['221111_144114__25660to28100'] = {
        'Raw':                      'volume/scanData/',
        'Smoothed':                 'volume/scanData/221111_144114__binary_onlylines25660to28100 rloess 0.09.ply',
        'Smoothed and Alligned':    'volume/scanData/',
        'Annotations': {
            (200, 530): 3,
            (590, 660): 2,
            (680, 740): 2,
            (790, 880): 2,
            (1080, 1110): 2,
            (1270, 1330): 2,
            (1380, 1430): 2,
            (1560, 1580): 2,
            (1660, 1700): 2,
        },
    }

In [ ]:
# 221111_144114__47178to50517 - A 2nd iteration scan - recently added
# flightpath: from class 2 to class 3

scans['221111_144114__47178to50517'] = {
        'Raw':                      'volume/scanData/',
        'Smoothed':                 'volume/scanData/221111_144114__binary_onlylines47178to50517 rloess 0.09.ply',
        'Smoothed and Alligned':    'volume/scanData/',
        'Annotations': {
            (2550, 3060): 3,
            (2400, 2480): 2,
            (2350, 2370): 2,
            (2270, 2300): 2,
            (1860, 1900): 2,
            (1540, 1620): 2,
            (1400, 1450): 2,
            (1170, 1200): 2,
            (1070, 1100): 2,
        },
    }

In [ ]:
# 221111_144114__86841to90142 - A 2nd iteration scan - recently added
# flightpath: from class 3 to class 2

scans['221111_144114__86841to90142'] = {
        'Raw':                      'volume/scanData/',
        'Smoothed':                 'volume/scanData/221111_144114__binary_onlylines86841to90142 rloess 0.09.ply',
        'Smoothed and Alligned':    'volume/scanData/',
        'Annotations': {
            (240, 640): 3,
            (730, 800): 2,
            (840, 910): 2,
            (950, 1080): 2,
            (1340, 1420): 2,
            (1670, 1740): 2,
            (1820, 1890): 2,
            (2130, 2170): 2,
            (2240, 2270): 2,
        },
    }

In [ ]:
# 221111_144114__114378to117271 - A 2nd iteration scan - recently added
# flightpath: from class 2 to class 3

scans['221111_144114__114378to117271'] = {
        'Raw':                      'volume/scanData/',
        'Smoothed':                 'volume/scanData/221111_144114__binary_onlylines114378to117271 rloess 0.09.ply',
        'Smoothed and Alligned':    'volume/scanData/',
        'Annotations': {
            (2130, 2630): 3,
            (1920, 2010): 2,
            (1790, 1890): 2,
            (1610, 1750): 2,
            (1310, 1380): 2,
            (1110, 1160): 2,
            (1010, 1050): 2,
            (860, 890): 2,
            (770, 800): 2,
        },
    }

Function to get erosion class of a given profile with the offset in scan in mind

In [ ]:
def get_erosion_class_based_on_x_coordinate(annotations_dict, x):
    for annotation in annotations_dict.keys():
        if x > annotation[0] and x < annotation[1]:
            if prediction_strategy == 'Detection':
                return 1 
            elif prediction_strategy == 'Classification': 
                return annotations_dict[annotation]
    return 0

Function to make cute annotation lines

In [ ]:
def annotation_line(ax, xmin, xmax, y, text, ytext = 0, linecolor = 'red', linewidth = 1, fontsize = 12):
    ax.annotate('',
                xy=(xmin, y),
                xytext=(xmax, y),
                xycoords='data',
                textcoords='data',
                arrowprops={'arrowstyle': '|-|',
                            'color': linecolor, 
                            'linewidth':linewidth
                           }
               )
    
    ax.annotate('', 
                xy=(xmin, y), 
                xytext=(xmax, y), 
                xycoords='data', 
                textcoords='data',
                arrowprops={'arrowstyle': '<->', 
                            'color': linecolor, 
                            'linewidth':linewidth
                           }
               )

    xcenter = xmin + (xmax - xmin) / 2
    
    if ytext == 0:
        ytext = y + (ax.get_ylim()[1] - ax.get_ylim()[0]) / 2

    ax.annotate(text, xy=(xcenter,ytext), ha='center', va='center', fontsize=fontsize)

Save the data from scans alongside the filepaths in the dictionaries. All scans are made to start from 0 on the X axis.

In [ ]:
for scan_data in scans.values():
    file = scan_data[data_prep_stage]
    data = PlyData.read(file)

    data['vertex']['x'] -= min(data['vertex']['x'])

    # indices_to_remove = data['vertex']['z'] == 0
    # # https://stackoverflow.com/a/24553551
    # x = np.delete(data['vertex']['x'], indices_to_remove)
    # y = np.delete(data['vertex']['y'], indices_to_remove)
    # z = np.delete(data['vertex']['z'], indices_to_remove)

    # data['vertex']['x'] = x
    # data['vertex']['y'] = y
    # data['vertex']['z'] = z

    scan_data['data'] = data

In [ ]:
depth_range = default_settings['depth']
figs_and_plots = []
plot_count = 0

for scan_data in scans.values():
    data = scan_data['data']

    x = data['vertex']['x']
    y = data['vertex']['y']
    z = data['vertex']['z']

    # print(len(np.unique(x)))

    figs_and_plots.append(plt.subplots(1, figsize=(48,8)))
    image = figs_and_plots[plot_count][1].scatter(x, y, c=z, s=0.1, vmin=depth_range[0], vmax=depth_range[1])
    figs_and_plots[plot_count][1].title.set_text('Annotated erosion on blade')
    figs_and_plots[plot_count][0].colorbar(image, ax=figs_and_plots[plot_count][1])
    # plt.xticks(
    #     ticks = np.arange(0, len(np.unique(x)), 20), 
    #     labels = list(map(int, np.unique(x)[::20])), 
    #     rotation = 90,
    # )

    for erosion_annotation in scan_data['Annotations'].keys():
        erosion_class = get_erosion_class_based_on_x_coordinate(
            scan_data['Annotations'], 
            erosion_annotation[0] + 1,
        )
        annotation_line(
            ax = figs_and_plots[plot_count][1], 
            text=str(erosion_class), 
            xmin=erosion_annotation[0], 
            xmax=erosion_annotation[1], 
            y=0, 
            ytext=10, 
            linewidth=3, 
            linecolor='red', 
            fontsize=16
        )
    # fig, axs = plt.subplots(1, 3, figsize=(16,9))

    # axs[0].boxplot(x)
    # axs[0].set_title('X Axis')

    # axs[1].boxplot(y)
    # axs[1].set_title('Y Axis')

    # axs[2].boxplot(z)
    # axs[2].set_title('Z Axis')

    plot_count += 1

Function to find an object with most subobjects in a list

In [ ]:
def find_max_length_list(l):
    list_len = [len(i) for i in l]
    return max(list_len)


Function to make all items the same length as the longest item in the list by filling it with zeros

In [ ]:
def pad_list_to_length_with(a, N, x = 0):
    return a + [x] * (N - len(a))


Function to evaluate predictions of a neural network from a lists of probabilities

In [ ]:
def evaluate_predictions(nn_predictions):
    nn_predictions = [np.argmax(pred) for pred in nn_predictions]
    return nn_predictions


Function to render Confusion Matrix and calculate Accuracy, Precision and Recall

In [ ]:
def model_scores(test_labels, predicted_labels):

    accuracy = accuracy_score(test_labels, predicted_labels)
    precision = precision_score(test_labels, predicted_labels, labels=test_labels, average="micro")
    recall = recall_score(test_labels, predicted_labels, labels=test_labels, average="micro")

    # print("Accuracy is:", accuracy)
    # print("Accuracy is the ratio of correct True Positives and True negatives,")
    # print("This metric is always iffy since in our case it would score almost 40% by always saying that there is no erosion.")
    # print()

    # print("Precision is:", precision)
    # print("Precision is the ratio of actual True Positives in the predicted Positives,")
    # print("In other words: how many items that we predicted to be a class are actually of that class?")
    # print()

    # print("Recall is:", recall)
    # print("Recall is the ratio of predicted True Positives of all True Positives,")
    # print("In other words: how many of all the True Positives did we find?")
    # print()

    # disp = ConfusionMatrixDisplay.from_predictions(test_labels, predicted_labels)
    # disp.ax_.set_title("Predicted values vs actual values of erosion class")

    return accuracy, precision, recall


Function to render predictions after modelling

In [ ]:
def postmodelling_render(model_type, fold, predictions):
    if postmodeling_renders != "Disabled" and fold == 1:
        for scan_data in scans.values():
            fig, ax = plt.subplots(1, figsize=(48,8))
            image = ax.scatter(x, y, c=z, s=0.1, vmin=depth_range[0], vmax=depth_range[1])
            ax.title.set_text(data_prep_stage + ' blade - ' + model_type + ' - Erosion ' + prediction_strategy + ', Fold #' + str(fold))
            fig.colorbar(image, ax=ax)
            
            for erosion_annotation in scan_data['Annotations'].keys():
                erosion_class = get_erosion_class_based_on_x_coordinate(
                    scan_data['Annotations'], 
                    erosion_annotation[0] + 1,
                )
                annotation_line(
                    ax = ax, 
                    text='Class ' + str(erosion_class), 
                    xmin=erosion_annotation[0], 
                    xmax=erosion_annotation[1], 
                    y=0, 
                    ytext=10,
                    linewidth=3, 
                    linecolor='red', 
                    fontsize=16
                )
            
            for i in range(len(train_labels_ids)):
                annotation_line(
                    ax = ax, 
                    text='', 
                    xmin=train_labels_ids[i]*11, 
                    xmax=train_labels_ids[i]*11 + 10, 
                    y=-298, 
                    ytext=-280, 
                    linewidth=3, 
                    linecolor='yellow', 
                    fontsize=16
                )
            
            for i in range(len(test_labels)):
                if predictions[i] == test_labels[i]:
                    annotation_line(
                        ax = ax, 
                        text=predictions[i], 
                        xmin=test_labels_ids[i]*11, 
                        xmax=test_labels_ids[i]*11 + 10, 
                        y=-298, 
                        ytext=-280, 
                        linewidth=3, 
                        linecolor='green', 
                        fontsize=16
                    )
                
                else:
                    annotation_line(
                        ax = ax, 
                        text=predictions[i], 
                        xmin=test_labels_ids[i]*11, 
                        xmax=test_labels_ids[i]*11 + 10, 
                        y=-298, 
                        ytext=-280, 
                        linewidth=3, 
                        linecolor='red', 
                        fontsize=16
                    )

Total number of profiles

In [ ]:
#len(np.unique(x))

Slicing

In [ ]:
max_length_list = 0
for scan_data in scans.values():
    scan_data['slice_start_coordinates'] = []
    scan_data['x_slices'] = []
    scan_data['y_slices'] = []
    scan_data['z_slices'] = []

    current_x = 0

    for xi in np.unique(scan_data['data']['vertex']['x'])[1:]:
        if xi > current_x + 10:
            current_x = xi
            # print(xi)
            scan_data['slice_start_coordinates'].append(xi)

    print("There are", len(scan_data['slice_start_coordinates']), "slices to be made")
    print(scan_data['slice_start_coordinates'])

    stop = scan_data['slice_start_coordinates'][-1]
    current_index = 0

    for coordinate in scan_data['slice_start_coordinates']:
        if coordinate == stop:
            break

        next_coordinate = scan_data['slice_start_coordinates'][current_index + 1]
        mask = (
            (scan_data['data']['vertex']['x'] >= coordinate) &
            (scan_data['data']['vertex']['x'] < next_coordinate)
        )

        scan_data['x_slices'].append(scan_data['data']['vertex']['x'][mask])
        scan_data['y_slices'].append(scan_data['data']['vertex']['y'][mask])
        scan_data['z_slices'].append(scan_data['data']['vertex']['z'][mask])
        
        current_index += 1

    # scan_data['cnnz_profiles'] = []
    # for profile_number in np.unique(scan_data['data']['vertex']['x']):
    #     mask = scan_data['data']['vertex']['x'] == profile_number
    #     scan_data['cnnz_profiles'].append(scan_data['data']['vertex']['z'][mask])

    # print(scan_data['cnnz_profiles'].shape)

    # scan_data['cnnz_profiles'] = np.array(scan_data['cnnz_profiles'])
    # scan_data['cnnz_profiles'] = np.array_split(scan_data['cnnz_profiles'], len(scan_data['slice_start_coordinates']))
    # scan_data['cnnz_profiles'] = np.array(scan_data['cnnz_profiles'])
 
    max_length_list = max(max_length_list, find_max_length_list(scan_data['z_slices']))
    # max_length_list = max(max_length_list, find_max_length_list(scan_data['cnnz_profiles']))

    # shape = scan_data['cnnz_profiles'].shape
    # scan_data['cnnz_profiles'] = scan_data['cnnz_profiles'].reshape((shape[0], shape[1], shape[3]))

Padding

In [ ]:
for scan_data in scans.values():
    scan_data['z_slices_padded'] = []
    for z_slice in scan_data['z_slices']:
        scan_data['z_slices_padded'].append(pad_list_to_length_with(z_slice.tolist(), max_length_list))

Storing mean erosion of each slice as future target for modeling

In [ ]:
for scan_data in scans.values():
    scan_data['targets'] = []
    for x_slice in scan_data['x_slices']:
        scan_data['targets'].append(get_erosion_class_based_on_x_coordinate(
            scan_data['Annotations'],
            mean(x_slice),
        ))
    print(scan_data['targets'])

Encoding each slice label with the number of that slice

In [ ]:
for scan_data in scans.values():
    for i in range(len(scan_data['targets'])):
        scan_data['targets'][i] = scan_data['targets'][i] + i * 10

    print(scan_data['targets'])

Merge of slices and tagets

In [ ]:
z_slices_padded = []
targets = []
for scan_data in scans.values():
    for slice in scan_data['z_slices_padded']:
        z_slices_padded.append(slice)
    for target in scan_data['targets']:
        targets.append(target)


Splitting the data onto train and test subsets

In [ ]:
# train_data, test_data, train_labels, test_labels = train_test_split(z_slices_padded, 
#                                                                     targets, 
#                                                                     test_size=0.33, 
#                                                                     #random_state=42
#                                                                 )


Isolating slice numbers and labels

In [ ]:
def isolate_slice_ids():
    train_labels_ids = np.zeros(len(train_labels), dtype=int)

    for i in range(len(train_labels)):
        train_labels_ids[i] = train_labels[i] / 10
        train_labels[i] = train_labels[i] % 10

    test_labels_ids = np.zeros(len(test_labels), dtype=int)

    for i in range(len(test_labels)):
        test_labels_ids[i] = test_labels[i] / 10
        test_labels[i] = test_labels[i] % 10
    
    return train_labels, test_labels, train_labels_ids, test_labels_ids

Scaling of the data

In [ ]:
def scaling():
    # min_max_scaler = MinMaxScaler()
    # min_max_scaler.fit(train_data)
    # train_data = min_max_scaler.transform(train_data)
    # test_data = min_max_scaler.transform(test_data)

    standard_scaler = StandardScaler()
    standard_scaler.fit(train_data)

    global standard_scaled_train_data
    standard_scaled_train_data = standard_scaler.transform(train_data)
    
    global standard_scaled_test_data
    standard_scaled_test_data = standard_scaler.transform(test_data)

    # type(train_data[0][0])

In [ ]:
def create_snn1_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(max_length_list,)),
        tf.keras.layers.Dense(4),
        tf.keras.layers.Softmax()
    ])
    return model

In [ ]:
def create_snn2_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(512, activation='relu', input_shape=(max_length_list,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Softmax()
    ])
    return model

In [ ]:
def create_cnn1_model():
    model = tf.keras.Sequential([
        # tf.keras.layers.Lambda(lambda x: x, input_shape=(max_length_list, 10, )),
        tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='linear', padding='same', input_shape=(max_length_list, 10)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Softmax()
    ])
    return model

In [ ]:
def update_metrics_scores(model, predictions):
    metrics_scores[model] = tuple(map(sum, zip(
        metrics_scores[model], 
        model_scores(test_labels, predictions,
    ))))

Modeling loop

In [ ]:
metrics_scores = {
    "Zero Benchmark":                       (0, 0, 0),
    "High Benchmark":                       (0, 0, 0),
    "Logistic Regression":                  (0, 0, 0),
    "Support Vector Machine":               (0, 0, 0),
    "Random Forest":                        (0, 0, 0),
    "Sequential Neural Network 1":          (0, 0, 0),
    "Sequential Neural Network 2":          (0, 0, 0),
    "Convolutional Neural Network 1":       (0, 0, 0),
}

num_folds = 4

kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

z_slices_padded = np.array(z_slices_padded)
targets = np.array(targets)

for fold, (train_indices, val_indices) in enumerate(kfold.split(z_slices_padded, targets)):

    print(f"Fold {fold + 1}:")
    print(f"train_indices type: {type(train_indices)}, val_indices type: {type(val_indices)}")

    train_data, test_data = z_slices_padded[train_indices], z_slices_padded[val_indices]
    print(train_data.shape)

    train_labels, test_labels = targets[train_indices], targets[val_indices]
    
    train_labels, test_labels, train_labels_ids, test_labels_ids = isolate_slice_ids()
    scaling()

    # Zero Benchmark
    zero_benchmark = len(test_labels)*[0]
    update_metrics_scores("Zero Benchmark", zero_benchmark)
    # postmodelling_render('Zero Benchmark', fold, zero_benchmark)

    # High Benchmark
    if prediction_strategy == 'Detection':
        top_class = 1
    elif prediction_strategy == 'Classification':
        top_class = 3
    high_benchmark = len(test_labels)*[top_class]
    update_metrics_scores("High Benchmark", high_benchmark)
    # postmodelling_render('High Benchmark', fold, high_benchmark)

    # Logistic Regression
    lr = LogisticRegression(random_state=0, max_iter=1000).fit(standard_scaled_train_data, train_labels)
    lr_predictions = lr.predict(standard_scaled_test_data)
    update_metrics_scores("Logistic Regression", lr_predictions)
    # postmodelling_render('Logistic Regression', fold, lr_predictions)

    # Support Vector Machine
    svc = svm.SVC(random_state=0, kernel="linear").fit(standard_scaled_train_data, train_labels)
    svc_predictions = svc.predict(standard_scaled_test_data)
    update_metrics_scores("Support Vector Machine", svc_predictions)
    # postmodelling_render('Support Vector Machine', fold, svc_predictions)

    # Random Forest
    rf = RandomForestClassifier().fit(train_data, train_labels)
    rf_predictions = rf.predict(test_data)
    update_metrics_scores("Random Forest", rf_predictions)
    postmodelling_render('Random Forest', fold, rf_predictions)

    # Sequential Neural Network 1
    model1 = create_snn1_model()
    model1.compile(optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=['accuracy'])
    
    model1.fit(train_data, train_labels, epochs=10)
    nn_predictions1 = model1.predict(test_data)
    nn_predictions1 = evaluate_predictions(nn_predictions1)
    update_metrics_scores("Sequential Neural Network 1", nn_predictions1)
    postmodelling_render('Sequential Neural Network 1', fold, nn_predictions1)

    # Sequential Neural Network 2
    model2 = create_snn2_model()
    model2.compile(optimizer='adam',
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=['accuracy'])
    
    model2.fit(train_data, train_labels, epochs=10)
    nn_predictions2 = model2.predict(test_data)
    nn_predictions2 = evaluate_predictions(nn_predictions2)
    update_metrics_scores("Sequential Neural Network 2", nn_predictions2)
    postmodelling_render('Sequential Neural Network 2', fold, nn_predictions2)

    # Convolutional Neural Network 1
    # cmodel1 = create_cnn1_model()
    # cmodel1.compile(optimizer='adam',
    #     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    #     metrics=['accuracy'])
    
    # cmodel1.fit(train_data, train_labels, epochs=10)
    # cnn_predictions1 = model1.predict(test_data)
    # cnn_predictions1 = evaluate_predictions(cnn_predictions1)
    # update_metrics_scores("Convolutional Neural Network 1", cnn_predictions1)
    # postmodelling_render('Convolutional Neural Network 1', fold, cnn_predictions1)

    K.clear_session()

for item in metrics_scores:
    metrics_scores[item] = tuple(round(x / num_folds, 2) for x in metrics_scores[item])

headers = ["Model Name", "Accuracy", "Precision", "Recall"]
array_of_arrays = np.array([[key] + list(value) for key, value in metrics_scores.items()])

fig, ax = plt.subplots()
ax.axis("off")
table = ax.table(cellText=array_of_arrays, colLabels=headers, cellLoc="center", loc="center", colColours=["#f0f0f0"] * len(headers))

table.auto_set_font_size(False)
table.set_fontsize(10)

table.auto_set_column_width([0, 1, 2, 3])

plt.show()